In [23]:
from imports import *

In [24]:
# Count available cores
cores = multiprocessing.cpu_count()
# In this case the amount of executors will be equal to the amount of cores
instances = cores

spark = SparkSession.builder \
          .appName("AMD-SM2L Joint Project") \
          .config("spark.driver.memory", "3g") \
          .config("spark.executor.memory", "4g") \
          .config("spark.executor.instances", cores) \
          .config("spark.executor.cores", cores//instances) \
          .config("spark.sql.shuffle.partitions", cores) \
          .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
          .config("spark.sql.execution.arrow.enabled", "true") \
          .getOrCreate()

spark.sparkContext.setLogLevel("OFF")
dataframe = spark.read.parquet("src/datasets/my_HI-Small_Trans.parquet", header=True)

# Computing

In [25]:
from MyGraph import *
train = spark.read.parquet("src/datasets/HI_Small_train_set.parquet", header=True)
test = spark.read.parquet("src/datasets/HI_Small_test_set.parquet", header=True)
dataframe = spark.read.parquet("src/datasets/HI-Small_features.parquet", header=True).select('id', 'same_account', 'same_bank', 'amount_difference', 'same_amounts', 'receiving_currency', 'payment_currency', 'same_currency', 'payment_format', 'is_laundering')

In [32]:
train_graph = MyGraph(train)
train_graph.get_forwards()
train_graph.same_or_similar()
train_graph.compute_fan()

/Users/simonemalesardi/spark/python/pyspark/sql/dataframe.py:169: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/Users/simonemalesardi/spark/python/pyspark/sql/dataframe.py:148: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [33]:
train_graph.join_ids()

train_graph.ids = train_graph.ids.drop('before_forward','forward','exists_same')#filter('fan_out_degree IS NOT NULL').show()
train_graph.df = train_graph.df.join(train_graph.ids, 'id', 'left').na.fill(value=0,subset=['exists_similar'])\
    .na.fill(value=1,subset=['fan_out_degree', 'fan_in_degree'])

train = train_graph.df.select('id',
 'week',
 'day_of_month',
 'hour',
 'amount_received',
 'amount_paid',
 'day_of_week',
 'transactions_same_hour_fa',
 'transactions_same_day_fa',
 'transactions_same_week_fa',
 'transactions_same_hour_fata',
 'transactions_same_day_fata',
 'transactions_same_week_fata',
 'from_account_inDegree',
 'from_account_outDegree',
 'to_account_inDegree',
 'to_account_outDegree',
 'exists_similar',
 'fan_in_degree',
 'fan_out_degree')

train = train.join(dataframe, 'id', 'left')
train.write.parquet("src/datasets/train_all_feats.parquet")

In [52]:
test_graph = MyGraph(test)
test_graph.get_forwards()
test_graph.same_or_similar()
test_graph.compute_fan()

In [58]:
test_graph.join_ids()

test_graph.ids = test_graph.ids.drop('before_forward','forward','exists_same')#filter('fan_out_degree IS NOT NULL').show()
test_graph.df = test_graph.df.join(test_graph.ids, 'id', 'left').na.fill(value=0,subset=['exists_similar'])\
    .na.fill(value=1,subset=['fan_out_degree', 'fan_in_degree'])

test = test_graph.df.select('id',
 'week',
 'day_of_month',
 'hour',
 'amount_received',
 'amount_paid',
 'day_of_week',
 'transactions_same_hour_fa',
 'transactions_same_day_fa',
 'transactions_same_week_fa',
 'transactions_same_hour_fata',
 'transactions_same_day_fata',
 'transactions_same_week_fata',
 'from_account_inDegree',
 'from_account_outDegree',
 'to_account_inDegree',
 'to_account_outDegree',
 'exists_similar',
 'fan_in_degree',
 'fan_out_degree')

test = test.join(dataframe, 'id', 'left')
test.write.parquet("src/datasets/test_all_feats.parquet")